In [19]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from keras_tuner.tuners import RandomSearch
from sklearn.model_selection import train_test_split

In [7]:
df=pd.read_csv('Real_Combine.csv')

In [8]:
X=df.iloc[:,:-1]
Y=df.iloc[:,-1]


In [16]:
def build_model(hp):
    model=Sequential()
    for i in range(hp.Int('num_layers',2,20)):
        model.add(Dense(units=hp.Int('units_'+str(i),min_value=32,max_value=512,step=32),activation='relu'))
    model.add(Dense(units=1,activation='linear'))
    model.compile(optimizer=Adam(hp.Choice('learning_rate',[1e-2, 1e-3, 1e-4])),
                 loss='mean_absolute_error',
                 metrics=['mean_absolute_error'])
    return model

In [17]:
tuner=RandomSearch(build_model,objective='val_mean_absolute_error',max_trials=5,executions_per_trial=3,directory='project',project_name='Air Quality Index')

In [18]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [20]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,Y,test_size=.3,random_state=0)

In [21]:
tuner.search(Xtrain,Ytrain,epochs=5,validation_data=(Xtest,Ytest))

Trial 5 Complete [00h 00m 05s]
val_mean_absolute_error: nan

Best val_mean_absolute_error So Far: nan
Total elapsed time: 00h 00m 21s
INFO:tensorflow:Oracle triggered exit


tuner.results_summary()

# I don't know why i am not getting th score nan